In [2]:
import os
import json
from pathlib import Path

PROFILE_CACHE = {}
base = Path(r"P:\VScode\Billing_Recon\Billing_Recon\profiles")

for partner_dir in base.iterdir():
    if partner_dir.is_dir():
        partner = partner_dir.name

        for version_dir in partner_dir.iterdir():
            if version_dir.is_dir():
                edi_version = version_dir.name
                profile_file = version_dir / "profile.json"

                if profile_file.exists():
                    key = (partner.upper(), edi_version)
                    PROFILE_CACHE[key] = json.loads(profile_file.read_text())
print(f"Loaded {len(PROFILE_CACHE)} partner profiles.")

print(PROFILE_CACHE.keys())

if PROFILE_CACHE.get("GLOBAL", 'Default') is None:
    print("GLOBAL profile not found.")

Loaded 5 partner profiles.
dict_keys([('CARRIER5010', '005010'), ('CARRIERX', '004010'), ('CARRIERY', '004010'), ('CODE', 'Charge_Dictionaries'), ('GLOBAL', 'default')])


In [17]:
import os

edi_text = """ISA*00*          *00*          *ZZ*CARRIERX       *ZZ*OURBROKER     *251101*1430*U*00401*000000001*0*T*>~ 

GS*IN*CARRIERX*OURBROKER*20251101*1430*1*X*004010~ 

ST*210*0001~ 

B3**INV1001*20251101*PP*BOL778231*CC~ 

REF*CN*PR0123456~ 

REF*LO*LD-56712~ 

N1*SH*XYZ CORP~ 

N1*CN*ABC DISTRIBUTION~ 

N1*BT*OURBROKER AP~ 

G62*11*20251030~ 

G62*70*20251102~ 

L1*1*2201.35***BASE FREIGHT~ 

L1*2*192.45***FUEL SURCHARGE~ 

L1*3*150.00***DETENTION~ 

L3*2543.80~ 

SE*15*0001~ 

GE*1*1~ 

IEA*1*000000001~"""

element_delim = '*'
segment_delim = '~'

#remove new line characters
edi_text = edi_text.replace('\n', '')
# print(edi_text)
segments = {}
segments_data = edi_text.strip().split(segment_delim) # for seg in edi_text.split(element_delim) if seg]
for seg in segments_data:
    elements = seg.split(element_delim)
    # print(elements)
    key = elements[0].strip()
    segments.setdefault(key, []).append(elements)
print(segments)
print(type(segments))
partner = segments['GS'][0][2].strip()
edi_version = segments['GS'][0][8].strip()
print(f"Partner: {partner}, EDI Version: {edi_version}")

{'ISA': [['ISA', '00', '          ', '00', '          ', 'ZZ', 'CARRIERX       ', 'ZZ', 'OURBROKER     ', '251101', '1430', 'U', '00401', '000000001', '0', 'T', '>']], 'GS': [[' GS', 'IN', 'CARRIERX', 'OURBROKER', '20251101', '1430', '1', 'X', '004010']], 'ST': [[' ST', '210', '0001']], 'B3': [[' B3', '', 'INV1001', '20251101', 'PP', 'BOL778231', 'CC']], 'REF': [[' REF', 'CN', 'PR0123456'], [' REF', 'LO', 'LD-56712']], 'N1': [[' N1', 'SH', 'XYZ CORP'], [' N1', 'CN', 'ABC DISTRIBUTION'], [' N1', 'BT', 'OURBROKER AP']], 'G62': [[' G62', '11', '20251030'], [' G62', '70', '20251102']], 'L1': [[' L1', '1', '2201.35', '', '', 'BASE FREIGHT'], [' L1', '2', '192.45', '', '', 'FUEL SURCHARGE'], [' L1', '3', '150.00', '', '', 'DETENTION']], 'L3': [[' L3', '2543.80']], 'SE': [[' SE', '15', '0001']], 'GE': [[' GE', '1', '1']], 'IEA': [[' IEA', '1', '000000001']], '': [['']]}
<class 'dict'>
Partner: CARRIERX, EDI Version: 004010


In [4]:
profile = PROFILE_CACHE.get(('CARRIERX', '004010'))

invoice_id_seg = profile['segments']['header']['invoice_id']

print(profile)

print(f"Invoice ID Segment: {invoice_id_seg}")

{'partner': 'CARRIERX', 'edi_version': '004010', 'profile_version': '1.0.0', 'envelope': {'detect': {'gs_sender': True, 'isa_sender': True}}, 'segments': {'header': {'invoice_id': {'seg': 'B3', 'idx': 2}, 'invoice_date': {'seg': 'B3', 'idx': 3, 'fmt': 'yyyyMMdd'}, 'bol': {'seg': 'B3', 'idx': 7}, 'pro': {'seg': 'REF', 'qual': 'CN', 'idx': 2}}, 'parties': [{'seg': 'N1', 'qual': 'SH', 'nameIdx': 2, 'mapTo': 'parties.ship_from'}, {'seg': 'N1', 'qual': 'CN', 'nameIdx': 2, 'mapTo': 'parties.ship_to'}, {'seg': 'N1', 'qual': 'BT', 'nameIdx': 2, 'mapTo': 'parties.bill_to'}], 'dates': [{'seg': 'G62', 'qual': '11', 'mapTo': 'dates.pickup', 'fmt': 'yyyyMMdd'}, {'seg': 'G62', 'qual': '70', 'mapTo': 'dates.delivery', 'fmt': 'yyyyMMdd'}], 'charges': {'strategy': 'L1_only', 'l1_rules': [{'contains': ['BASE FREIGHT', 'LINEHAUL'], 'mapTo': 'charges.base_freight'}, {'contains': ['FSC', 'FUEL SURCHARGE'], 'mapTo': 'charges.fuel_surcharge'}, {'contains': ['DETENTION'], 'mapTo': 'charges.detention'}, {'defa

In [39]:
header = profile['segments']['header']
print(f"Header Segment: {header}")
parties = profile['segments']['parties']
print(f"Parties Segment: {parties}")
dates = profile['segments']['dates']
print(f"Dates Segment: {dates}")
charges = profile['segments']['charges']
print(f"Charges Segment: {charges}")
total_rules = profile['segments']['total']
print(f"Totals Segment: {total_rules}")
currency_rules = profile['currency']
print(f"Currency Segment: {currency_rules}")
    

Header Segment: {'invoice_id': {'seg': 'B3', 'idx': 2}, 'invoice_date': {'seg': 'B3', 'idx': 3, 'fmt': 'yyyyMMdd'}, 'bol': {'seg': 'B3', 'idx': 7}, 'pro': {'seg': 'REF', 'qual': 'CN', 'idx': 2}}
Parties Segment: [{'seg': 'N1', 'qual': 'SH', 'nameIdx': 2, 'mapTo': 'parties.ship_from'}, {'seg': 'N1', 'qual': 'CN', 'nameIdx': 2, 'mapTo': 'parties.ship_to'}, {'seg': 'N1', 'qual': 'BT', 'nameIdx': 2, 'mapTo': 'parties.bill_to'}]
Dates Segment: [{'seg': 'G62', 'qual': '11', 'mapTo': 'dates.pickup', 'fmt': 'yyyyMMdd'}, {'seg': 'G62', 'qual': '70', 'mapTo': 'dates.delivery', 'fmt': 'yyyyMMdd'}]
Charges Segment: {'strategy': 'L1_only', 'l1_rules': [{'contains': ['BASE FREIGHT', 'LINEHAUL'], 'mapTo': 'charges.base_freight'}, {'contains': ['FSC', 'FUEL SURCHARGE'], 'mapTo': 'charges.fuel_surcharge'}, {'contains': ['DETENTION'], 'mapTo': 'charges.detention'}, {'defaultOther': True}], 'sac_rules': []}
Totals Segment: {'seg': 'L3', 'idx': 1, 'fallbackToSum': True}
Currency Segment: {'default': 'USD'

In [40]:
invoice_id_rule = header['invoice_id']
print(f"Invoice ID rule: {invoice_id_rule}")
invoice_date_rule = header['invoice_date']
print(f"Invoice Date rule: {invoice_date_rule}")
bol_rule = header['bol']
print(f"BOL rule: {bol_rule}")
pro_rule = header['pro']
print(f"PRO rule: {pro_rule}")

Invoice ID rule: {'seg': 'B3', 'idx': 2}
Invoice Date rule: {'seg': 'B3', 'idx': 3, 'fmt': 'yyyyMMdd'}
BOL rule: {'seg': 'B3', 'idx': 7}
PRO rule: {'seg': 'REF', 'qual': 'CN', 'idx': 2}


In [41]:
for party in parties:
    if party['mapTo'] == "parties.ship_from":
        ship_from_rule = party
        print(f"Ship From rule: {ship_from_rule}")
    elif party['mapTo'] == "parties.ship_to":
        ship_to_rule = party
        print(f"Ship To rule: {ship_to_rule}")
    elif party['mapTo'] == "parties.bill_to":
        bill_to_rule = party
        print(f"Bill To rule: {bill_to_rule}")

Ship From rule: {'seg': 'N1', 'qual': 'SH', 'nameIdx': 2, 'mapTo': 'parties.ship_from'}
Ship To rule: {'seg': 'N1', 'qual': 'CN', 'nameIdx': 2, 'mapTo': 'parties.ship_to'}
Bill To rule: {'seg': 'N1', 'qual': 'BT', 'nameIdx': 2, 'mapTo': 'parties.bill_to'}


In [61]:
for date in dates:
    if date['mapTo'] == "dates.pickup":
        pickup_date_rule = date
        print(f"Ship Date rule: {pickup_date_rule}")
    elif date['mapTo'] == "dates.delivery":
        delivery_date_rule = date
        print(f"Delivery Date rule: {delivery_date_rule}")

Ship Date rule: {'seg': 'G62', 'qual': '11', 'mapTo': 'dates.pickup', 'fmt': 'yyyyMMdd'}
Delivery Date rule: {'seg': 'G62', 'qual': '70', 'mapTo': 'dates.delivery', 'fmt': 'yyyyMMdd'}


In [69]:
if charges['strategy'] == "L1_only" or charges['strategy'] == "L1_then_SAC":
    l1_rules = charges['l1_rules']
    sac_rules = None
    print(f"L1 Rules: {l1_rules}")
    if charges['strategy'] == "L1_then_SAC":
        sac_rules = charges['sac_rules']
        print(f"SAC Rules: {sac_rules}")
elif charges['strategy'] == "SAC_only":
    l1_rules = None
    sac_rules = charges['sac_rules']
    print(f"SAC Rules: {sac_rules}")

L1 Rules: [{'contains': ['BASE FREIGHT', 'LINEHAUL'], 'mapTo': 'charges.base_freight'}, {'contains': ['FSC', 'FUEL SURCHARGE'], 'mapTo': 'charges.fuel_surcharge'}, {'contains': ['DETENTION'], 'mapTo': 'charges.detention'}, {'defaultOther': True}]


In [44]:
our_broker = 'OURBROKER'

invoice_id = segments[invoice_id_rule['seg']][0][invoice_id_rule['idx']].strip()
print(f"Invoice ID: {invoice_id}")
if our_broker == segments['GS'][0][3].strip():
    side = 'Buy'
else:
    side = 'Sell'
print(f"Side: {side}")
source = { "type": "edi210", "doc_uri": "null"}
print(f"Source: {source}")

Invoice ID: INV1001
Side: Buy
Source: {'type': 'edi210', 'doc_uri': 'null'}


In [45]:
carrier = {"name": "null", "scac": "null"}
print(f"Carrier: {carrier}")
customer = {"name": "null", "account_id": "null"}
print(f"Customer: {customer}")

Carrier: {'name': 'null', 'scac': 'null'}
Customer: {'name': 'null', 'account_id': 'null'}


In [ ]:
bol = segments[bol_rule['seg']][0][bol_rule['idx']].strip()
print(f"BOL: {bol}")

In [50]:
pro = segments[pro_rule['seg']][0][pro_rule['idx']].strip()
print(f"PRO: {pro}")

PRO: PR0123456


In [51]:
po = "null"
print(f"PO: {po}")
load_id = segments[pro_rule['seg']][1][2].strip()
print(f"Load ID: {load_id}")

PO: null
Load ID: LD-56712


In [59]:
for N1_seg in segments[parties[0]['seg']]:
    if N1_seg[1] == "SH":
        ship_from = N1_seg[ship_from_rule['nameIdx']].strip()
    elif N1_seg[1] == "CN":
        ship_to = N1_seg[ship_to_rule['nameIdx']].strip()
    elif N1_seg[1] == "BT":
        bill_to = N1_seg[bill_to_rule['nameIdx']].strip()
print(f"Ship From: {ship_from}")
print(f"Ship To: {ship_to}")
print(f"Bill To: {bill_to}")


Ship From: XYZ CORP
Ship To: ABC DISTRIBUTION
Bill To: OURBROKER AP


In [ ]:
for date in segments[dates[0]['seg']]:
    if date[1] == "11":
        pickup = date[2].strip()
    elif date[1] == "70":
        delivery = date[2].strip()
print(f"Pickup Date: {pickup}")
print(f"Delivery Date: {delivery}")       

Pickup Date: 20251030
Delivery Date: 20251102


In [ ]:
if l1_rules:
    base_freight = 0.00
    fuel_surcharge = 0.00
    detention = 0.00
    other = []
    # print(l1_rules)
    # print(segments['L1'])
    for l1_rule in l1_rules:
        # print(f"L1 Rule: {l1_rule}")
        mapto = l1_rule.get('mapTo','charges.other').split('.')[-1]
        # print(f"MapTo: {mapto}")
        contains = l1_rule.get('contains', [])
        # print(f"Contains: {contains}")
        for l1_charge in segments['L1']:
            # print(f"L1 Charge: {l1_charge}")
            if any(c in l1_charge for c in contains):
                if mapto == "base_freight":
                    base_freight += float(l1_charge[2].strip())
                    # print(f"Base Freight updated: {base_freight}")
                elif mapto == "fuel_surcharge":
                    fuel_surcharge += float(l1_charge[2].strip())
                    # print(f"Fuel Surcharge updated: {fuel_surcharge}")
                elif mapto == "detention":
                    detention += float(l1_charge[2].strip())
                    # print(f"Detention updated: {detention}")
                elif mapto == "other":
                    other.append({
                        "code": l1_charge[2].strip(),
                        "desc": l1_charge[-1].strip(),
                        "amount": float(l1_charge[2].strip())
                    })
print(f"Base Freight: {base_freight}")
print(f"Fuel Surcharge: {fuel_surcharge}")
print(f"Detention: {detention}")
print(f"Other Charges: {other}")

Base Freight: 2201.35
Fuel Surcharge: 192.45
Detention: 150.0
Other Charges: []


In [89]:
sum_total = 0.00
sum_total += base_freight
sum_total += fuel_surcharge
sum_total += detention
for oth in other:
    sum_total += oth['amount']
print(f"Sum Total from Charges: {sum_total}")
total = float(segments[total_rules['seg']][0][total_rules['idx']].strip())
print(f"Total from EDI: {total}")

Sum Total from Charges: 2543.7999999999997
Total from EDI: 2543.8


In [90]:
currency = currency_rules['default']
print(f"Currency: {currency}")

Currency: USD
